# Simulação de Parâmetros de Estoque com Visualização em Cards

Este notebook calcula os principais parâmetros de controle de estoque e apresenta os resultados de forma visual com explicações acessíveis.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from IPython.display import display, Markdown

In [ ]:
# 🧮 Função para calcular os parâmetros logísticos
def calcular_parametros(demanda_diaria, desvio_padrao_diario, lead_time_dias, custo_pedido, custo_armazenagem, preco_unitario):
    z = 1.65  # 95% de nível de serviço
    es = z * desvio_padrao_diario * np.sqrt(lead_time_dias)
    estoque_max = es + (demanda_diaria * lead_time_dias)

    # LEC limitado ao estoque máximo
    demanda_anual = demanda_diaria * 250
    lec = np.sqrt((2 * demanda_anual * custo_pedido) / (custo_armazenagem * preco_unitario))
    lec = min(lec, estoque_max)

    # Recalcula intervalo baseado no LEC ajustado
    intervalo = lec / demanda_diaria

    # Estoque médio condicionado
    if lec >= estoque_max:
        estoque_medio = es + ((demanda_diaria * lead_time_dias) / 2)
    else:
        estoque_medio = es + (lec / 2)

    # PP = ponto de pedido
    pp = es + (demanda_diaria * lead_time_dias) / 2
    if pp > estoque_medio:
        estoque_medio = pp  # garantia que EM ≥ PP

    return {
        "ES": round(es, 2),
        "PP": round(pp, 2),
        "EMAX": round(estoque_max, 2),
        "EM": round(estoque_medio, 2),
        "LEC": round(lec, 2),
        "IPD": round(intervalo, 2)
    }

In [ ]:
# 🖼️ Função para gerar imagem em estilo "cards"
def gerar_cards_parametros(dados):
    fig, ax = plt.subplots(figsize=(12, 4))
    ax.set_xlim(0, 6)
    ax.set_ylim(0, 1)
    ax.axis('off')

    cores = ['#FF6666', '#FFB266', '#66CC99', '#66B2FF', '#C299FF', '#FF99C8']
    siglas = list(dados.keys())
    valores = list(dados.values())

    for i, (sigla, valor) in enumerate(zip(siglas, valores)):
        card = Rectangle((i, 0), 1, 1, edgecolor='black', facecolor=cores[i], linewidth=1.5)
        ax.add_patch(card)
        ax.text(i + 0.5, 0.65, sigla, ha='center', va='center', fontsize=13, fontweight='bold')
        ax.text(i + 0.5, 0.35, f"{valor}", ha='center', va='center', fontsize=12)

    plt.tight_layout()
    plt.savefig("cards_simulacao_estoque.png")
    plt.show()

In [ ]:
# 🏁 Execução do exemplo prático
resultado = calcular_parametros(
    demanda_diaria=40,
    desvio_padrao_diario=8,
    lead_time_dias=5,
    custo_pedido=120,
    custo_armazenagem=0.20,
    preco_unitario=30
)

# Exibir parâmetros calculados
for k, v in resultado.items():
    print(f"- {k}: {v}")

# Gerar imagem final
gerar_cards_parametros(resultado)

In [ ]:
# 📘 Explicações simples para leigos
explicacoes = {
    "ES": f" Seu estoque mínimo de segurança deve ser de {resultado['ES']} unidades para evitar rupturas.",
    "PP": f" Quando seu estoque atingir {resultado['PP']} unidades, é hora de fazer um novo pedido.",
    "EMAX": f" O máximo de produtos que você deve manter é {resultado['EMAX']} unidades.",
    "EM": f" Seu estoque médio ao longo do tempo será de aproximadamente {resultado['EM']} unidades.",
    "LEC": f" O lote ideal de compra é de {resultado['LEC']} unidades para equilibrar custos.",
    "IPD": f" Você fará um novo pedido a cada {resultado['IPD']} dias em média."
}

for explicacao in explicacoes.values():
    display(Markdown(f"- {explicacao}"))